In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torchvision.datasets as dataset # for loading dataset (mnist)
import torchvision.transforms as transforms # for processing datasets
from torch.utils.data import DataLoader # for making dataset easier to use 

from matplotlib import pyplot as plt
import numpy as np

import random

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: gpu") if torch.cuda.is_available() else print("device: cpu")

device: gpu


In [2]:
# hypter parameter setting
epochs = 200
learning_rate = 1e-2
display_step = 20
batch_size = 1024
# momentum = 0.9
dropout_rate = 0.25

In [3]:
# load data
torchvision_transform = transforms.Compose([
    transforms.Resize((36, 36)), 
    transforms.RandomCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

train_data = dataset.CIFAR10("./", train = True, transform = torchvision_transform, target_transform = None, download = True)
test_data = dataset.CIFAR10("./", train = False, transform = transforms.ToTensor(), target_transform = None, download = True)

# check the data
print('len(train_data): ', len(train_data))
print('len(test_data): ', len(test_data))

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True, num_workers = 1, drop_last = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = True, num_workers = 1, drop_last = True)

Files already downloaded and verified
Files already downloaded and verified
len(train_data):  50000
len(test_data):  10000


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__() # for initializing nn.Module (parent class)
        self.feature_extraction = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Conv2d(32, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.MaxPool2d(2,2),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.MaxPool2d(2,2),

            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Conv2d(512, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.MaxPool2d(2,2),

            nn.Conv2d(256, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Conv2d(128, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
        )

        self.classifier = nn.Sequential(
            nn.Linear(64 * 4 * 4, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(100, 10),
        )
     
    def forward(self, x):
        output = self.feature_extraction(x)

        flatten = output.view(batch_size, -1)
        result = self.classifier(flatten)
        return result

model = CNN().to(device)
model.train()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [5]:
loss_array = []

# train the model
for i in range(epochs):
    lr = learning_rate * (0.1 ** (i // 100))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    for index, [data, label] in enumerate(train_loader):
        data = data.to(device)
        label = label.to(device)
                
        optimizer.zero_grad()
        output = model.forward(data)
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        
    if i % display_step == 0:
        print('{} epoch loss: {}'.format(i,loss))
        loss_array.append(loss.cpu().detach().numpy())

        #test the model
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for index, [data, label] in enumerate(test_loader):
                data = data.to(device)
                label = label.to(device)
                
                output = model.forward(data)
                _, prediction_index = torch.max(output, 1)
                
                total += label.size(0)
                correct += (prediction_index == label).sum().float()

            if total == 0:
                total = 1
            print("Accuracy of the model: {}".format(correct/total))

        model.train()

0 epoch loss: 1.8045344352722168
Accuracy of the model: 0.1955295205116272
20 epoch loss: 0.32153743505477905
Accuracy of the model: 0.800889790058136
40 epoch loss: 0.1851670742034912
Accuracy of the model: 0.8539496660232544
60 epoch loss: 0.13235488533973694
Accuracy of the model: 0.8645833134651184
80 epoch loss: 0.047368984669446945
Accuracy of the model: 0.8814018964767456
100 epoch loss: 0.012992381118237972
Accuracy of the model: 0.88671875
120 epoch loss: 0.012830565683543682
Accuracy of the model: 0.8908420205116272
140 epoch loss: 0.008919920772314072
Accuracy of the model: 0.8865017294883728
160 epoch loss: 0.01624266244471073
Accuracy of the model: 0.8909505009651184
180 epoch loss: 0.009657550603151321
Accuracy of the model: 0.8986892294883728


In [6]:
#test the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for index, [data, label] in enumerate(test_loader):
        data = data.to(device)
        label = label.to(device)
        
        output = model.forward(data)
        _, prediction_index = torch.max(output, 1)

        
        total += label.size(0)
        correct += (prediction_index == label).sum().float()

    print("Accuracy of the model: {}".format(correct/total))

Accuracy of the model: 0.9016102695465088


In [7]:
# 모멘텀 0.9일때 52에포스에 59%
# 모멘텀 0.8로 수정 후 재시작 36에포스에 65%가 최고기록
# 드롭아웃 제거, 데이터 증강법 적용 54에포스에 68% 기록
# 매 에포스마다 데이터 증강법 적용되도록 수정 16에포스에 67% 기록
# 모멘텀 0.7로 수정 16에포스에 61%기록 다시 0.8로 복구
# 러닝레이트 1e-4로 수정, 데이터증강법 시도한거 다 삭제. 잘못되어 있엇음
# cnn모델 각 스테이지별 블록 개수 2->3으로 증가 57%가 끝
# learning_rate = 1e-1, momentum=0.9, weight_decay=5e-4로 변경 각스테이지별 블록 개수 3->2로 복구   22에 62달성 일단 중단
# 학습주기별 러닝레이트 변경 기능 추가 learning_rate_plan = [[50, 1e-1], [50, 1e-2], [50, 1e-3]] 로 시도. 첫단계에서 아에 안내려감
# learning_rate_plan = [[50, 1e-2], [50, 1e-3], [50, 1e-4]]로 시도 70%가 끝
# 배치 사이즈 128에서 32로 축소, 아주간단한 모델로 변경 -> 79% 달성
# 배치사이즈 그대로, 원래 복잡한 모델로 변경 14->78%,  82%달성
# 간단한 모델로 변경, 소프트 맥스 추가, 아담으로 변경, learning_rate_plan = [[20, 1e-4], [20, 1e-5], [20, 1e-6], [20, 1e-7]] 84%
# 드롭아웃 2개중 1개꼴로 추가 83%
# 드롭아웃 전부다 추가, 드롭아웃에 relu실수로 들어가있던거 다 삭제 10에 80.3% 중단
# 64 128에 한층씩 축소 6에 79.3% 줄였지만 큰 차이 없음. 중단
# 소프트맥스에도 dropout추가, 512 하나 추가 12에 81.9%
# 복잡한 모델  8에 74% 탈락
# 간단한 모델 더 빠른 학습을 위해 learning_rate_plan = [[10, 1e-3], [10, 1e-4], [20, 1e-5]] 로 변경 20에 85%로 중단
# 10, 1e-3에서 10은 너무 짧았던것 같음. 10에 82.2%로끝났는데 로스는 이전까지 절반으로 줄어두는 중이었음
# learning_rate_plan = [[20, 1e-3], [20, 1e-4], [20, 1e-5]] 로 적용, 끝에 512->256 추가 14에 82.5%
# 256 -> 128 추가 8에 82.9%
# 128 -> 64 추가 24에 84.7%
# learning_rate_plan = [[20, 1e-1], [20, 1e-2], [20, 1e-3], [20, 1e-4], [20, 1e-5]] 로 변경, 소프트 맥스 제거, eval호출 후 test데이터 측정 후 train호출 추가
# 러닝레이트 적용 제대로 된걸로 수정 후 복잡한 모델. 
# 복잡한 모델 삭제. 비교상대가 안된다.
# 배치 사이즈 줄인다고 성능 향상이 되는것은 아니다. 빨라진만큼 러닝레이트를 늘려주면 정확도는 비슷한데 속도는 더 빠르다.
# 맥스풀링간의 간격이 너무 좁으면 안된다.
# 맥스풀링간의 간격을 2개에서 3개로 늘림. 74에서 89.69% 달성.
# 끝에 3->16, 64->32 추가
# 로스가 0.0x인 것이 0.0000x인 것보다 좋기도 하다. 로스가 적을수록 또는 어느정도 이하일때 항상 좋은게 아니다.
# 3->16, 64->32 삭제 에포스 단계별 100으로 증가